In [1]:
# Import from the globals_and_helpers file
from globals_and_helpers import (
    PROJECT_DIR,
    MODELS_DIR,
    ORIG_IMAGE_DIR,
    TEMP_OUTPUTS_DIR,
    FINAL_OUTPUTS_DIR,
    TILE_SIZE,
    BASE_NAMES,
    normalize_image,
    plot_histogram,
    get_dapi_and_collagen_paths,
    convert_seconds_to_hms,
    get_base_name,
    check_image_dimensions,
    gen_STIFMap_tile_path,
    save_stiffness_colormap,
    stitch_images,
)

In [2]:
from STIFMaps import STIFMap_generation
from STIFMaps.misc import get_step

import os
import re
import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy import interpolate
from PIL import Image
import tifffile
import time

# STIFMap models
models = [
    os.path.join(MODELS_DIR, 'iteration_1171.pt'),
    os.path.join(MODELS_DIR, 'iteration_1000.pt'),
    os.path.join(MODELS_DIR, 'iteration_1043.pt'),
    os.path.join(MODELS_DIR, 'iteration_1161.pt'),
    os.path.join(MODELS_DIR, 'iteration_1180.pt')
]

# Parameters
STIFMap_SCALE_FACTOR = 0.4
# STIFMap_SCALE_FACTOR = 2.712
STIFMap_STEP = get_step(40, STIFMap_SCALE_FACTOR)
STIFMap_SQUARE_SIDE = get_step(224, STIFMap_SCALE_FACTOR)
STIFMap_BATCH_SIZE = 100

print('Step size is ' + str(STIFMap_STEP) + ' pixels')
print('Side length for a square is ' + str(STIFMap_SQUARE_SIDE) + ' pixels')

Step size is 100 pixels
Side length for a square is 560 pixels


In [3]:
# Function to generate and save the STIFMap
def gen_STIFMap_tile(dapi_path, collagen_path, name, step, models, batch_size, square_side, check_existing=True):
    output_path = gen_STIFMap_tile_path(dapi_path)
    
    # Check if the tile has already been processed
    if check_existing and os.path.exists(output_path):
        print(f"Skipping already processed tile: {dapi_path} and {collagen_path}")
        return

    start_time = time.perf_counter()

    z_out = STIFMap_generation.generate_STIFMap(
        dapi=dapi_path, 
        collagen=collagen_path, 
        name=name, 
        step=step, 
        models=models,
        mask=False, 
        batch_size=batch_size, 
        square_side=square_side,
        save_dir=False
    )

    end_time = time.perf_counter()
    print("Elapsed time:", convert_seconds_to_hms(end_time - start_time))

    output_image = np.mean(z_out, axis=0)

    global_min = np.min(output_image)
    global_max = np.max(output_image)
    output_image_normalized = (output_image - global_min) / (global_max - global_min)

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    plt.imsave(output_path, output_image_normalized, cmap="viridis")
    print(f"Saved image: {output_path}")

    # Save the raw stiffness values as a NumPy array
    stiffness_values_path = output_path.replace(".png", ".npy")
    np.save(stiffness_values_path, output_image)
    print(f"Saved stiffness values: {stiffness_values_path}")

# Function to check if a tile has already been processed
def is_tile_completed(output_path):
    return os.path.exists(output_path)

# dapi_path, collagen_path = get_dapi_and_collagen_paths(
#     base_name=BASE_NAMES[0], orig_image_dir=ORIG_IMAGE_DIR
# )
# print(f"dapi_path={dapi_path}, collagen_path={collagen_path}")
# gen_STIFMap_tile(
#     # dapi_path=dapi_path,
#     # collagen_path=collagen_path,
#     dapi_path='/home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/27620/IPMN_tiles/27620_C0_0_0.tif',
#     collagen_path='/home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/27620/IPMN_tiles/27620_C1_0_0.tif',
#     name='test',
#     step=STIFMap_STEP,
#     models=models,
#     batch_size=STIFMap_BATCH_SIZE,
#     square_side=STIFMap_SQUARE_SIDE,
#     check_existing=True  # Set to True to check if the file already exists
# )

In [ ]:
def get_base_file_name(file_path):
    # Extract the file name without the extension
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    # Keep only the first two parts (e.g., '27620_C0') by splitting on '_'
    base_name = "_".join(file_name.split("_")[:2])
    return base_name

# Generate the STIFMaps for all the tiles for a base image
def gen_STIFMap(base_name):
    dapi_path, collagen_path = get_dapi_and_collagen_paths(base_name, orig_image_dir=ORIG_IMAGE_DIR)
    base_name_C0 = get_base_file_name(dapi_path)
    base_name_C1 = get_base_file_name(collagen_path)    
    
    # Automatically determine num_rows and num_cols based on filenames
    tile_pattern_C0 = re.compile(rf"{base_name_C0}_(\d+)_(\d+)\.tif")
    tile_pattern_C1 = re.compile(rf"{base_name_C1}_(\d+)_(\d+)\.tif")

    # tile_pattern = re.compile(rf"{base_name_C0}_(\d+)_(\d+)\.tif")
    tile_image_dir = os.path.join(TEMP_OUTPUTS_DIR, base_name, "IPMN_tiles")
    
    # Dictionary to store how many columns exist per row
    row_col_map = {}
    
    # Loop through files and match patterns
    for file in os.listdir(tile_image_dir):
        # match = tile_pattern.match(file)
        match_C0 = tile_pattern_C0.match(file)
        match_C1 = tile_pattern_C1.match(file)

        if match_C0 or match_C1:
            match = match_C0 or match_C1
            row, col = map(int, match.groups())
            row_col_map.setdefault(row, set()).add(col)
            # print(f"Matched Row: {row}, Column: {col}")
    
    # Find the maximum number of columns
    if not row_col_map:
        raise ValueError("No matching files found in the directory.")
    
    max_cols = max(len(cols) for cols in row_col_map.values())
    
    # Select only rows with the full set of columns
    valid_rows = sorted([row for row, cols in row_col_map.items() if len(cols) == max_cols])
    num_rows = len(valid_rows)
    num_cols = max_cols

    # print("Row to Columns mapping:", row_col_map)
    # print("Valid rows with full columns:", valid_rows)
    
    print(f"Detected grid size: {num_rows} rows x {num_cols} columns")
    
    # Create a list of file names for C0 and C1 tiles
    # C0_files = [f"{base_name_C0}_{i}_{j}.tif" for i in range(num_rows) for j in range(num_cols)]
    # C1_files = [f"{base_name_C1}_{i}_{j}.tif" for i in range(num_rows) for j in range(num_cols)]
    
    # Main: Loop through all the tiled C0 and C1 images and pass each one to run_STIFMap()
    for row in valid_rows:  # Iterate only over rows with full columns
        for col in range(num_cols):
            dapi_path = os.path.join(tile_image_dir, f"{base_name_C0}_{row}_{col}.tif")
            collagen_path = os.path.join(tile_image_dir, f"{base_name_C1}_{row}_{col}.tif")
    
            # Ensure files exist before processing
            if not os.path.exists(dapi_path) or not os.path.exists(collagen_path):
                print(f"Skipping missing tile: {dapi_path} or {collagen_path}")
                continue
            
            output_path = gen_STIFMap_tile_path(dapi_path)
            # output_path = gen_output_path(dapi_path)
            
            if is_tile_completed(output_path):
                print(f"Skipping already processed tile: {dapi_path} and {collagen_path}")
                continue

            print(f"Generating STIFMap for {dapi_path}, {collagen_path}")
            gen_STIFMap_tile(
                dapi_path, collagen_path, name=base_name, step=STIFMap_STEP,
                models=models, batch_size=STIFMap_BATCH_SIZE, square_side=STIFMap_SQUARE_SIDE,
                check_existing=True
            )
    
    stitched_output_path = os.path.join(FINAL_OUTPUTS_DIR, f"{base_name}_STIFMap_stitched.png")
    stitch_images(stitched_output_path, base_name, image_format='png')
    save_stiffness_colormap(stitched_output_path, base_name)
            
gen_STIFMap(base_name=BASE_NAMES[0])

Detected grid size: 6 rows x 9 columns
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_0_0.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_0_0.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 45
Num squares in y direction is 45
Time taken to predict squares is 23.014177560806274
Time taken to predict squares is 42.78710746765137
Time taken to predict squares is 62.34855937957764
Time taken to predict squares is 82.26913189888
Time taken to predict squares is 102.34050369262695
Total time taken is 103.402911901474
Elapsed time: 1 minutes, 43.4 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_0_0.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_0_0.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_0_1.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_0_1.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 45
Num squares in y direction is 45
Time taken to predict squares is 20.136373281478882
Time taken to predict squares is 39.95189952850342
Time taken to predict squares is 60.05371451377869
Time taken to predict squares is 79.87101602554321
Time taken to predict squares is 100.67441415786743
Total time taken is 101.62596368789673
Elapsed time: 1 minutes, 41.6 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_0_1.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_0_1.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_0_2.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_0_2.tif
Device is cpu


...

Image shape is (3, 5003, 5003)
Num squares in x direction is 45
Num squares in y direction is 45
Time taken to predict squares is 19.518123149871826
Time taken to predict squares is 39.052465200424194
Time taken to predict squares is 58.447494983673096
Time taken to predict squares is 78.01000308990479
Time taken to predict squares is 97.34577512741089
Total time taken is 98.27940011024475
Elapsed time: 1 minutes, 38.3 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_2_6.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_2_6.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_2_7.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_2_7.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 45
Num squares in y direction is 45
Time taken to predict squares is 19.61232566833496
Time taken to predict squares is 39.13421821594238
Time taken to predict squares is 58.64314103126526
Time taken to predict squares is 78.3181939125061
Time taken to predict squares is 97.78459525108337
Total time taken is 98.94150972366333
Elapsed time: 1 minutes, 39.0 seconds
Saved image: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_2_7.png
Saved stiffness values: /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/STIFMap_tiles/1865_2_7.npy
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_2_8.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_2_8.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 5003, 5003)
Num squares in x direction is 45
Num squares in y direction is 45
Time taken to predict squares is 19.423487424850464
Time taken to predict squares is 38.80644392967224
Time taken to predict squares is 58.369614362716675
Time taken to predict squares is 77.77997183799744


In [ ]:
for base_name in BASE_NAMES:
    gen_STIFMap(base_name=base_name)

Row to Columns mapping: {2: {0, 1, 2}, 1: {0, 1, 2}, 0: {0, 1, 2}}
Valid rows with full columns: [0, 1, 2]
Detected grid size: 3 rows x 3 columns
Generating STIFMap for /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C0_0_0.tif, /home/steve/Projects/WeaverLab/STIFMaps/temp_outputs/1865/IPMN_tiles/1865_C1_0_0.tif
Device is cpu


/home/steve/.local/lib/python3.10/site-packages/STIFMaps/STIFMap_generation.py:145: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  io.imshow(np.moveaxis(im, 0, -1))


Image shape is (3, 20000, 20000)
Num squares in x direction is 245
Num squares in y direction is 245
Time taken to predict squares is 463.0807526111603
Time taken to predict squares is 936.1929988861084


In [43]:
# Stitch the processed tiles back together
# stitched_output_path = os.path.join(STIFMaps_directory, f"{base_name[0]}_STIFMap_stitched_v2.png")
# base_name = BASE_NAMES[0]
# stitched_output_path = os.path.join(TEMP_DIR, f"{base_name}_STIFMap_stitched.png")
# stitch_images(stitched_output_path, base_name, image_format='png')

Detected grid size: 15 rows x 18 columns
Missing tile: 27620_STIFMap_13_3.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_4.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_5.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_6.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_7.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_8.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_9.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_10.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_11.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_12.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_13.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_14.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_15.png. Replacing with a white tile.
Missing tile: 27620_STIFMap_13_16.png. Replacing with a white tile.
Missing tile: 